# Занятие 4. Проверка гипотез

Практические вопросы, которые решает статистика, включают в себя например, действительно ли отличаютс между собой две выборки. Действительно ли среднее значение в выборке отличается от некоторого значения. Правда ли мальчики пишут экзамены по математике лучше, чем девочки.

Все эти вопросы решаются с помощью проверки гипотез.

## Проверка гипотез

### Гипотеза

Гипотеза — это предположение о параметрах распределения.

Выделяют две гипотезы: нулевая и альтернативная.

Нулевая гипотеза $H_0$ — это гипотеза о том, что параметры распределения равны некоторому значению. Альтернативная гипотеза $H_1$ — это гипотеза о том, что параметры распределения не равны некоторому значению.

В случае сравнения двух выборок, нулевая гипотеза — это гипотеза о том, что средние значения двух выборок равны. Альтернативная гипотеза — это гипотеза о том, что средние значения двух выборок не равны.

Посмотрим на примерах.


In [ ]:
import pandas as pd

data = pd.read_csv("StudentsPerformance.csv")
data

In [ ]:
import matplotlib.pyplot as plt

male_math = data[data.gender == "male"]["math score"]
female_math = data[data.gender == "female"]["math score"]

plt.hist(male_math, alpha=0.4, range=(0, 100))
plt.hist(female_math, alpha=0.4, range=(0, 100));

In [ ]:
import numpy as np

def compare_distributions(d1, d2, label1, label2):
    plt.figure(figsize=(10, 2))

    plt.scatter(d1, np.zeros(len(d1)), color="blue", alpha=0.1)
    plt.scatter(d2, np.ones(len(d2)) - 0.5, color="red", alpha=0.1)

    mean1 = np.mean(d1)
    mean2 = np.mean(d2)

    plt.vlines(mean1, 0, 0.5, label=f"mean {label1}", color="blue")
    plt.vlines(mean2, 0, 0.5, label=f"mean {label2}", color="red")

    plt.yticks(ticks=[0, 0.5], labels=[label1, label2])
    plt.xlabel("Оценка")

    plt.legend()
    plt.show()

compare_distributions(male_math, female_math, "boys", "girls")


### Ошибки первого и второго рода

Ошибка первого рода (ложноположительный результат) — это ситуация, когда мы отвергаем нулевую гипотезу, хотя она верна. Ошибка второго рода (ложноотрицательный результат) — это ситуация, когда мы принимаем нулевую гипотезу, хотя она неверна.

### Пример в нашем случае.

Нулевая гипотеза $H_0$ --  средние оценки мальчиков и девочек совпадают.

Альтернативная гипотеза $H_1$ --  средние оценки мальчиков и девочек не совпадают.

Ошибка первого рода (ложноположительный результат) -- мы решили, что оценки не совпадают, хотя они совпадают.

Ошибка второго рода (ложноотрицательный результат) -- мы решили, что оценки совпадают, хотя они не совпадают.


### t-test Стьюдента

Один из самых простых и распространенных тестов для проверки гипотез о среднем значении. Его формула: 

$$t = \frac{\bar{x} - \mu_0}{\sigma / \sqrt{n}}$$

где $\bar{x}$ — среднее значение выборки, $\mu_0$ — значение, которое мы сравниваем с средним значением выборки, $\sigma$ — стандартное отклонение выборки, $n$ — размер выборки.


Посмотрим, как эта статистика меняется, если постепенно отдалять распределение от проверяемого средего значения.

In [ ]:
import scipy.stats as sps

def t_test(data, mu_0):
    mean = data.mean()
    std = data.std()
    count = len(data)
    t = (mean - mu_0) / (std / np.sqrt(count))
    return t

mean_0 = 0
sigma = 1

samples = []
t_test_results = []
mean_range = np.linspace(-5, 5, 100)
for m in mean_range:
    data = sps.norm(m, sigma).rvs(1000)
    t_test_results.append(t_test(data, mean_0))


In [ ]:
import matplotlib.pyplot as plt

plt.plot(mean_range, t_test_results)
plt.xlabel("Отклонение от заданного среднего")
plt.ylabel("Значение статистики")
plt.plot()

Из теории статистики известно, что результаты t-теста при верной нулевой гипотезе порождены распределением Стьюдента с `n - 1` степенями свободы (параметр распределения), где `n` -- это размер выборки. 

Сначала просто посмотрим как выглядит распределение Стьюдента.

In [ ]:
degrees_of_freedom = [1, 2, 5, 15]

X = np.linspace(-4, 4, 1000)


plt.figure(figsize=(8, 5))
plt.grid()
for d in degrees_of_freedom:
    density = sps.t(df=d).pdf(X)
    plt.plot(X, density, label=f"df = {d}")

norm = sps.norm.pdf(X)
plt.plot(X, norm, label="normal", linewidth=2)
plt.xlabel("x")
plt.ylabel("density")
plt.legend()

plt.show()

Теперь проверим, что действительно результаты т-теста порождены распределением Стьюдента. Для этого возьмём много (7000) выборок нормального распределения со средним в нуле. Дальше посчитаем для каждой выборки т-тест. И посмотрим на гистограмме, как распределены результаты.

In [ ]:
mean_0 = 0
sigma = 1
sample_size = 1000

samples = []
t_test_results = []
for _ in range(7000):
    data = sps.norm(mean_0, sigma).rvs(sample_size)
    t_test_results.append(t_test(data, mean_0))

In [ ]:
plt.figure(figsize=(10, 6))
plt.hist(t_test_results, density=True, bins=40, label="Результаты t-теста")
plt.plot(np.linspace(-4, 4, 1000), sps.t(df=sample_size-1).pdf(np.linspace(-4, 4, 1000)), label="Распределение Стьюдента")
plt.xlabel("Значение статистики")
plt.ylabel("density")
plt.legend()
plt.show()

Для контраста, давайте посмотрим, где будет располагаться значение t-теста для выборки, у которой среднее равно не 0 а 0.1

In [ ]:
data = sps.norm(0.11, sigma).rvs(sample_size)
t_result = t_test(data, mean_0)

plt.figure(figsize=(10, 6))
plt.hist(t_test_results, density=True, bins=40, label="Результаты t-теста")
plt.plot(np.linspace(-4, 4, 1000), sps.t(df=sample_size-1).pdf(np.linspace(-4, 4, 1000)), label="Распределение Стьюдента")
plt.vlines(t_result, 0, 0.24, label="Результат со средним 0.11", color="red")
plt.xlabel("Значение статистики")
plt.ylabel("density")
plt.legend()
plt.show()



### p-value

Для численной оценки вероятности ошибки первого рода используется p-value. p-value -- это то, насколько вероятно получить данный или более экстремальный результат статистики (т-теста) при условии, что нулевая гипотеза верна. 

Если такая вероятность очень мала -- мы можем отвергнуть нулевую гипотезу. 

p-value считается с помощью функции распределения `F` для распределения Стьюдента.

Но вообще, в scipy уже реализован t-test, который автоматически считает статистику и p-value.

Однако, для наглядности, давайте нанесём p-value = 0.05 на предыдущий график.

In [ ]:
alpha = 0.05

l_pvals = sps.t(df=sample_size-1).ppf(alpha)
r_pvals = sps.t(df=sample_size-1).ppf(1 - alpha)

data = sps.norm(0.11, sigma).rvs(sample_size)
t_result = t_test(data, mean_0)

plt.figure(figsize=(10, 6))
plt.hist(t_test_results, density=True, bins=40, label="Результаты t-теста")
plt.plot(np.linspace(-4, 4, 1000), sps.t(df=sample_size-1).pdf(np.linspace(-4, 4, 1000)), label="Распределение Стьюдента")
plt.vlines(l_pvals, 0, 0.3, label="Левый p-value=0.05", color="black", linestyle=":")
plt.vlines(r_pvals, 0, 0.3, label="Правый p-value=0.05", color="red" , linestyle=":")
plt.vlines(t_result, 0, 0.3, label="Результат со средним 0.11", color="green")
plt.xlabel("Значение статистики")
plt.ylabel("density")
plt.legend()
plt.show()

### Виды гипотез

**Левосторонняя гипотеза**

Среднее значение выборки меньше некоторого значения.

**Правосторонняя гипотеза**

Среднее значение выборки больше некоторого значения.

**Двусторонняя гипотеза**

Среднее значение выборки не равно некоторому значению.

### Виды статистических тестов (на примере t-теста)

**Одновыборочный t-тест**

Формула:

$$t = \frac{\bar{x} - \mu_0}{\sigma / \sqrt{n}}$$

Проверяется гипотеза о том, что среднее значение выборки равно некоторому референсному значению.

**Двухвыборочный t-тест**

Формула:

$$t = \frac{\bar{x}_1 - \bar{x}_2}{\sqrt{\frac{\sigma_1^2}{n_1} + \frac{\sigma_2^2}{n_2}}}$$

Проверяется гипотеза о том, что средние значения двух выборок равны.

**Двухвыборочный t-тест с зависимыми выборками**

Формула:

$$
d = x_1 - x_2 \\ 
t = \frac{\bar{d} - 0}{\sigma_d / \sqrt{n}}
$$

Допустим, в двух экспериментах поучаствовала одна и та же группа людей. Тогда, чтобы проверить различие между экспериментами, мы можем поэлементно вычесть результаты первого эксперимента из результатов второго (для каждого человека), а потом применить к разности одновыборочный t-тест с нулевой гипотезой о том, что среднее значение разности равно 0.

### Одновыборочный t-тест

Реализован в модуле `scipy.stats` как `ttest_1samp`.

Аргумент `a` -- это выборка, по которой проверяется гипотеза.

Аргумент `popmean` -- это референсное значение, которое проверяется в гипотезе.

Вид гипотезы указывается в аргументе `alternative`.

* `alternative = "two-sided"` -- двусторонняя гипотеза
* `alternative = "less"` -- левосторонняя гипотеза
* `alternative = "greater"` -- правосторонняя гипотеза


Проведём такой эксперимент: 

1. Посчитаем среднюю оценку по математике среди всех учеников.
2. Проверим, отличается ли средняя оценка по математике среди тех, кто готовится к экзамену, от средней оценки по математике среди всех учеников.



In [ ]:
data = pd.read_csv("StudentsPerformance.csv")

general_mean = data["math score"].mean()

prepared_sample = data[data["test preparation course"] == "completed"]["math score"]

In [ ]:
plt.hist(prepared_sample)
plt.vlines(general_mean, 0, 80, color="red", label="general mean")
plt.vlines(prepared_sample.mean(), 0, 80, color="black", label="prepared mean")
plt.legend()
plt.xlabel("Math score")
plt.ylabel("frequency")
plt.show()

In [ ]:
print(f"Двусторонняя альтернатива:   {sps.ttest_1samp(a=prepared_sample, popmean=general_mean, alternative='two-sided')}")
print(f"Левосторонняя альтернатива:  {sps.ttest_1samp(a=prepared_sample, popmean=general_mean, alternative='less')}")
print(f"Правосторонняя альтернатива: {sps.ttest_1samp(a=prepared_sample, popmean=general_mean, alternative='greater')}")

### Двухвыборочный t-тест с независимыми выборками

Реализован в модуле `scipy.stats` как `ttest_ind`.

Аргументы функции:

* `a` -- первая выборка
* `b` -- вторая выборка
* `equal_var` -- флаг, указывающий, следует ли считать дисперсии равными
* `alternative` -- вид альтернативной гипотезы (двусторонняя, левосторонняя, правосторонняя, как в случае выше)

Наконец-то мы узнаем, является ли различие в оценках по математике у мальчиков и девочек статистически значимым.

Сначала руками проверим, равны ли дисперсии.

In [ ]:
male_math = data[data["gender"] == "male"]["math score"]
female_math = data[data["gender"] == "female"]["math score"]

male_var = male_math.var()
female_var = female_math.var()

print(f"Дисперсия у мальчиков: {round(male_var, 3)}")
print(f"Дисперсия у девочек: {round(female_var, 3)}")

In [ ]:
print(f"Равные дисперсии:{sps.ttest_ind(male_math, female_math, equal_var=True, alternative='greater')}")
print(f"Неравные дисперсии:{sps.ttest_ind(male_math, female_math, equal_var=False, alternative='greater')}")

### Двухвыборочный t-тест с зависимыми выборками

Реализован в модуле `scipy.stats` как `ttest_rel`.

Аргументы функции:

* `a` -- первая выборка
* `b` -- вторая выборка
* `alternative` -- вид альтернативной гипотезы (двусторонняя, левосторонняя, правосторонняя, как в случае выше)

Допустим, мы хотим проверить есть ли разница в оценках по письму и чтению у **одних и тех же** девочек. То есть две выборки зависимы, потому что это одни и те же ученики.

In [ ]:
writing_scores = data[data.gender == "female"]["writing score"]
reading_scores = data[data.gender == "female"]["reading score"]

print(f"Двусторонняя альтернатива:   {sps.ttest_rel(writing_scores, reading_scores, alternative='two-sided')}")
print(f"Левосторонняя альтернатива:  {sps.ttest_rel(writing_scores, reading_scores, alternative='less')}")
print(f"Правосторонняя альтернатива: {sps.ttest_rel(writing_scores, reading_scores, alternative='greater')}")

In [ ]:
plt.hist(writing_scores, range=(0, 100), bins=25, alpha=0.4)
plt.hist(reading_scores, range=(0, 100), bins=25, alpha=0.4)
plt.show()